%matplotlib inline
import matplotlib.pyplot as plt
import random
import pickle
import re
import pandas as pd# EDA (Easy Data Augmentation)
* SR (synonym replacement): 유의어로 바꾸기
* RI (random insertion): 임의의 단어 삽입
* RS (random swap): 특정 두 단어의 위치 바꾸기 - 이거 어카지
* RD (random deletion): 임의의 단어 삭제

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import pickle
import re
import pandas as pd
import numpy as np

In [20]:
import warnings

warnings.filterwarnings(action='ignore') 

KAIST에서 만든 유의어 데이터셋 wordnet 사용 <br>
https://github.com/catSirup/KorEDA 에서 가져온 코드

In [4]:
wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText

In [5]:
########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

In [6]:
########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

In [7]:
########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


In [8]:
########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

In [15]:
def EDA(df, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=2):
    eda_df = pd.DataFrame()
        
    for idx, sentence in enumerate(df['sentence']):
        sentence = get_only_hangul(sentence)
        words = sentence.split(' ')
        words = [word for word in words if word != ""]
        num_words = len(words)

        augmented_sentences = []
        num_new_per_technique = int(num_aug/4) + 1

        n_sr = max(1, int(alpha_sr*num_words))
        n_ri = max(1, int(alpha_ri*num_words))
        n_rs = max(1, int(alpha_rs*num_words))

        # # sr
        # for _ in range(num_new_per_technique):
        # 	a_words = synonym_replacement(words, n_sr)
        # 	a_sent = ' '.join(a_words)
        # 	if a_sent != sentence and a_sent not in augmented_sentences:
        # 		augmented_sentences.append(a_sent)

        #ri
        # for _ in range(num_new_per_technique):
        # 	a_words = random_insertion(words, n_ri)
        # 	a_sent = ' '.join(a_words)
        # 	if a_sent != sentence and a_sent not in augmented_sentences:
        # 		augmented_sentences.append(a_sent)

        # rs
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            a_sent = ' '.join(a_words)
            if a_sent != sentence and a_sent not in augmented_sentences:
                augmented_sentences.append(a_sent)

        #rd
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, p_rd)
            a_sent = ' '.join(a_words)
            if a_sent != sentence and a_sent not in augmented_sentences:
                augmented_sentences.append(a_sent)

        augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
        random.shuffle(augmented_sentences)

        if num_aug >= 1:
            augmented_sentences = augmented_sentences[:num_aug]
        else:
            keep_prob = num_aug / len(augmented_sentences)
            augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
            
        for s in augmented_sentences:
            eda_df = eda_df.append(pd.DataFrame([[int(df[['id']].loc[idx]), s, '', '', df[['label']].loc[idx][0], df[['source']].loc[idx][0]]], columns=list(df.columns)))

        # augmented_sentences.append(sentence)

    return eda_df


In [16]:
def under_data(df,num):
    under_data = pd.DataFrame()
    for k,v in df['label'].value_counts().items():
        if v < num:
            under_key = df[df['label'] == k]
            under_data = pd.concat([under_data, under_key])
    under_data.reset_index(drop = True, inplace = True)
    return under_data

In [17]:
# entity index 찾아서 생성된 데이터들을 저장한 df 생성
def create_augmented_df(df, sb):
    eda_df = pd.DataFrame()
    unmatch = 0
    for i in range(len(sb)):
        se_dict = {}
        oe_dict = {}

        # subject entity, object entity
        temp_df = data[data.id==sb.iloc[i]['id']]
        se = eval(temp_df.iloc[0]['subject_entity'])['word']
        oe = eval(temp_df.iloc[0]['object_entity'])['word']

        # back_translation에서의 인덱스 위치 구하기
        ## subject entity
        se_start_idx = sb.loc[i]['sentence'].find(se)

        if se_start_idx == -1:
            # print("subject entity에 해당하는 단어가 없습니다.")
            unmatch += 1
            continue

        se_end_idx = se_start_idx + len(se)
        if se != sb.iloc[i]['sentence'][se_start_idx:se_end_idx]:
            print("오류: 인덱스가 잘못 설정됨.")
            break
        else:
            se_dict['word'] = se
            se_dict['start_idx'] = se_start_idx
            se_dict['end_idx'] = se_end_idx
            se_dict['type'] = eval(df.loc[0]['subject_entity'])['type']
        ## object entity
        oe_start_idx = sb.loc[i]['sentence'].find(oe)
        if oe_start_idx == -1:
            # print("object entity에 해당하는 단어가 없습니다.")
            unmatch += 1
            continue

        oe_end_idx = oe_start_idx + len(oe)
        if oe != sb.loc[i]['sentence'][oe_start_idx:oe_end_idx]:
            print("오류: 인덱스가 잘못 설정됨.")
            # print(se, eda_list[i][se_start_idx:se_end_idx])
            break
        else:
            oe_dict['word'] = oe
            oe_dict['start_idx'] = oe_start_idx
            oe_dict['end_idx'] = oe_end_idx
            oe_dict['type'] = eval(df.loc[i%len(df)]['object_entity'])['type']

        sb.loc[i, 'subject_entity'] = str(se_dict)
        sb.loc[i, 'object_entity'] = str(oe_dict)
        eda_df = eda_df.append(pd.DataFrame([[int(temp_df[['id']].iloc[0]), sb.iloc[i]['sentence'], str(se_dict), str(oe_dict), temp_df[['label']].iloc[0][0],temp_df[['source']].iloc[0][0]]], columns=list(temp_df.columns)))

    return eda_df

In [10]:
data = pd.read_csv("../../dataset/train/train.csv")

중복데이터 없애는 거 어디갔냐

In [11]:
relation_data = data[data['label'] != "no_relation"]

In [12]:
print(f"총 데이터 개수: {data.shape[0]}\nrelation_data 개수: {relation_data.shape[0]}")

총 데이터 개수: 32470
relation_data 개수: 22936


In [13]:
relation_data['label'].value_counts()

org:top_members/employees              4284
per:employee_of                        3573
per:title                              2103
org:member_of                          1866
org:alternate_names                    1320
per:origin                             1234
org:place_of_headquarters              1195
per:date_of_birth                      1130
per:alternate_names                    1001
per:spouse                              795
per:colleagues                          534
per:parents                             520
org:founded                             450
org:members                             420
per:date_of_death                       418
org:product                             380
per:children                            304
per:place_of_residence                  193
per:other_family                        190
per:place_of_birth                      166
org:founded_by                          155
per:product                             139
per:siblings                    

In [18]:
u1000_data = under_data(relation_data, 1000)

In [19]:
u1000_data['label'].value_counts()

per:spouse                             795
per:colleagues                         534
per:parents                            520
org:founded                            450
org:members                            420
per:date_of_death                      418
org:product                            380
per:children                           304
per:place_of_residence                 193
per:other_family                       190
per:place_of_birth                     166
org:founded_by                         155
per:product                            139
per:siblings                           136
org:political/religious_affiliation     98
per:religion                            96
per:schools_attended                    82
org:dissolved                           66
org:number_of_employees/members         48
per:place_of_death                      40
Name: label, dtype: int64

In [22]:
ex_df = EDA(u1000_data)
ex_df.reset_index(drop = True, inplace = True)
ex_df

,id,sentence,subject_entity,object_entity,label,source
0,22,"첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수를 지낸 ...",,,per:spouse,wikipedia
1,22,박흥식은 첫 부인과의 사이에 장녀 박병숙을 지냈다. 두 번째 부인은 경희대학교 교수...,,,per:spouse,wikipedia
2,25,"SK는 또 ""해당 방송에서 언급한 지난해 방송 또한 명백한 허위사실임을 알려드린다""...",,,per:spouse,wikitree
3,25,"언급한 또 ""해당 방송에서 SK는 지난해 12월5일 알려드린다""며 가로세로연구소 방...",,,per:spouse,wikitree
4,33,채리나는 박용근을 생명의 은인으로 여기고 교제하다가 2016년 11월에 결혼하였다.,,,per:spouse,wikipedia
...,...,...,...,...,...,...
9498,29266,묵돌이 진나라가 된 209년에는 진시황이 사망한 직후였으며 이로 인해 선우가 큰 혼...,,,per:place_of_death,wikipedia
9499,30008,"만네르헤임은 1951년 1월 27일(협정 세계시), 핀란드 시간으로는 1월 28일 ...",,,per:place_of_death,wikipedia
9500,30008,"만네르헤임은 1951년 1월 27일(협정 1월 핀란드 시간으로는 세계시), 28일 ...",,,per:place_of_death,wikipedia
9501,30987,6월 2차 진주성 진주성이 함락되자 의병장인 김천일·고종후와 함께 촉석루에 올라 임...,,,per:place_of_death,wikitree


In [23]:
print(f"생성된 문장 개수: {len(ex_df)}\nex)\n{ex_df[:7]}")

생성된 문장 개수: 9503
ex)
   id                                           sentence subject_entity  \
0  22  첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수를 지낸 ...                  
1  22  박흥식은 첫 부인과의 사이에 장녀 박병숙을 지냈다. 두 번째 부인은 경희대학교 교수...                  
2  25  SK는 또 "해당 방송에서 언급한 지난해 방송 또한 명백한 허위사실임을 알려드린다"...                  
3  25  언급한 또 "해당 방송에서 SK는 지난해 12월5일 알려드린다"며 가로세로연구소 방...                  
4  33     채리나는 박용근을 생명의 은인으로 여기고 교제하다가 2016년 11월에 결혼하였다.                  
5  33  채리나는 이후 박용근을 생명의 은인으로 여기고 교제하다가 11월에 2016년 결혼하였다.                  
6  40            배우 장신영(35) 품에 남편 강경준(36) 씨가 둘째를 씨와 안았다.                  

  object_entity       label     source  
0                per:spouse  wikipedia  
1                per:spouse  wikipedia  
2                per:spouse   wikitree  
3                per:spouse   wikitree  
4                per:spouse  wikipedia  
5                per:spouse  wikipedia  
6                per:spouse   wikitree  


In [ ]:
result = create_augmented_df(u1000_data, aug_data)
result

In [36]:
augmented_data = pd.concat([data, result])

In [39]:
augmented_data.to_csv("./EDA(39680).csv")

In [155]:
# 걍 예시임
print(ex_sent)
print('-'*50)
for a_s in ex_aug_sents:
    if a_s != ex_sent:
        print(a_s)
        print('\n')

이듬해 1997년에는 부상에서 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 때린 뒤 1루수인 기요하라 가즈히로와 부딪혀 왼쪽 무릎에 타박상을 입었다.
--------------------------------------------------
이듬해 1997년에는 부상에서 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 때린 뒤 1루수인 기요하라 가즈히로와 부딪혀 무릎에 타박상을 입었다.


무릎에 1997년에는 부상에서 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 때린 뒤 1루수인 타박상을 가즈히로와 부딪혀 왼쪽 이듬해 기요하라 입었다.


이듬해 땅볼을 부상에서 회복된 채로 시즌을 맞이했으나 7월 무릎에 요미우리전에서 내야 1997년에는 때린 뒤 1루수인 기요하라 가즈히로와 부딪혀 왼쪽 1일 타박상을 입었다.


이듬해 때린 부상에서 입었다. 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 1997년에는 뒤 1루수인 기요하라 가즈히로와 부딪혀 왼쪽 무릎에 타박상을 회복된


이듬해 1997년에는 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 뒤 1루수인 기요하라 가즈히로와 부딪혀 왼쪽 무릎에 타박상을 입었다.


이듬해 1997년에는 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에서 내야 땅볼을 때린 뒤 1루수인 기요하라 가즈히로와 부딪혀 왼쪽 무릎에 타박상을 입었다.




In [62]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [63]:
tokenizer.pad_token_id

0

In [64]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModel